# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Proyecto Integrador**

### Tecnológico de Monterrey

### Prof Dra. Grettel Barceló Alonso y Dr. Luis Eduardo Falcón Morales

## Avance III de Proyecto

## Modelo Base

## Integrantes del Equipo:
### - Erika Cardona Rojas            A01749170
### - Miriam Bönsch                  A01330346
### - Mardonio Manuel Román Ramírez  A01795265

In [1]:
# Librerias
import pandas as pd
import yaml
import numpy as np

# Cargando Yaml
with open("../config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

In [2]:
# Cargando Base de Datos
df = pd.read_csv(r"../Data/DF_Final_No_Filtered.csv")

# Filtrado De Variables

Dada nuestra previa entrega, se ha discutido con los expertos la viabilidad de eliminación de las variables propuestas.

A continuación se aplicará la eliminación correspondiente a su retroalimentación.

In [3]:
# Creando PAD_media a partir de PAD_izq y PAD_der
df['PAD_media'] = df[['PAD_izq', 'PAD_der']].mean(axis=1)

In [4]:
# Eliminando variables correlacionadas y con un varianza casi nula.

vars_delete = config['Filtrado_Variables']['Umbral_Varianza'] + config['Filtrado_Variables']['Correlacionadas']

vars_delete = list(set(vars_delete))

df = df.drop(vars_delete, axis='columns')
print(f"Dada nuestra conversación con nuestros compañeros de Barcelona, se eliminarán {len(vars_delete)} variables.")

Dada nuestra conversación con nuestros compañeros de Barcelona, se eliminarán 93 variables.


# Reestructura De Datos

Los datos se encuentran repitiendo a cada sujeto 4 veces. Pues cada registro se divide en `Treatment` (Intervención y Control) y `Time` (Pre y Post).

Nos interesa conocer los efectos del experimento en el periodo Post de la Intervención con tomate.

Por lo que se estructurará la base de datos, de manera que nos funcione así.

In [5]:
X = df.drop(['id','grupo','Treatment','Time'], axis='columns')